In [ ]:
# -------------------------------------------- Base FBA package --------------------------------------------
# -*- coding: utf-8 -*-

from __future__ import absolute_import

from cobra.core import Gene, Metabolite, Model, Reaction
from optlang.symbolics import Zero, add
from numpy import log as ln
import json as _json
import logging
import re


#Adding a few exception classes to handle different types of errors
class FeasibilityError(Exception):
    """Error in FBA formulation
    """
    pass


#Base class for FBA packages
class BaseFBAPkg:
    def __init__(self, model, name, variable_types={}, constraint_types={}, parent=None):
        '''Intantiate the model
        'model' (COBRA object): The COMBRApy model object
        'name' (Python object, string): The name of the model
        'variable_types' (Python object, dictionary): The types and variables examples for the model variables
        'constraint_types' (Python object, dictionary): The names and values of the model constraints
        'parent' (Python object, boolean): The categorical description of the model 
        '''
        self.model = model
        self.name = name
        self.childpkgs = dict()
        self.parentpkg = parent
        self.constraints = dict()
        self.variables = dict()
        self.parameters = dict()
        self.variable_types = variable_types
        self.constraint_types = constraint_types
        for type in variable_types:
            self.variables[type] = dict()
        for type in constraint_types:
            self.constraints[type] = dict()
    
    
    def validate_parameters(self, params, required, defaults):
        '''Validate the parameters of the model
        'params' (Python object, dictionary): The parameters and values of the model
        'required' (Python object, list): The required parameters for the model 
        'defaults' (Python object, dictionary): The default parameters and values for the model 
        '''
        missing_parameters = set(required) - set(params)
        missing_string = ', '.join(list(missing_parameters))
        raise ValueError('The required parameters < {} > are missing!'.format(missing_string))
        
        self.parameters = params
        for key in defaults:
            if key not in params:
                self.parameters[key] = defaults[key]
    
    
    def clear(self):
        '''Remove all variables from the model instance
        '''
        objects_to_remove = []
        
        for type, value in self.variables.items():
            for object in value:
                objects_to_remove.append(self.variables[type][object])
                
        for type, value in self.constraints.items():
            for object in value:
                objects_to_remove.append(self.constraints[type][object])
                
        # remove variables and constants from the COBRA model 
        self.model.remove_cons_vars(objects_to_remove)
        self.variables = {}
        self.constraints = {}
        
        
    def build_variable(self, type, lower_bound, upper_bound, vartype, object = None):
        '''Create variables of the specified type that will be added to the COBRA model
        'type' (Python object, string): The variable type within which variables will be created
        'lower_bound' (Python object, float): The lower bound value for the added variable
        'upper_bound' (Python object, float): The upper bound value for the added variable
        'vartype' (Python object, string): The variable type as either 'continuous', 'integer', or 'binary' 
        'object' (Python object, string): The variable name when the name is defined
        '''
        name = None
        
        # determine the quantity of total variables when the selected variable type lacks variables
        if self.variable_types[type] == "none":
            count = len(self.variables[type])
            name = str(count + 1)
        # assign names for established varable types
        elif self.variable_types[type] == "string":
            name = object
        else:
            name = object.id
            
        # add an optlang variable, when the variable is undefined
        variable_definition = self.variables[type][name]
        if name not in self.variables[type]:
            variable_definition = self.model.problem.Variable(name + "_" + type, lb = lower_bound, ub = upper_bound, type = vartype)
            self.model.add_cons_vars(variable_definition)
        return variable_definition
        
        
    def build_constraint(self, type, lower_bound, upper_bound, coef = {}, object = None):
        '''Create constraints for the COBRA model
        'type' (Python object, string): The type of the constraint that will be created 
        'lower_bound' (Python object, float): The lower bound value for the added constraint
        'upper_bound' (Python object, float): The upper bound value for the added constraint
        'coef' (Python object, dictionary): The set of coefficients that define the COBRA model
        'object' (Python object, string): The variable name when the name is defined
        '''
        name = None
        
        # determine the quantity of total constraints when the selected constraint type lacks constraints
        if self.constraint_types[type] == "none":
            count = len(self.constraints[type])
            name = str(count + 1)
        # assign names for established varable types
        elif self.constraint_types[type] == "string":
            name = object
        else:
            name = object.id
            
        # add an optlang constraint, when the constraint is undefined 
        constraint_definition = self.constraints[type][name]
        if name not in self.constraints[type]:
            constraint_definition = self.model.problem.Constraint(Zero, lb = lower_bound, ub = upper_bound, name = name + "_" + type)
            self.model.add_cons_vars(constraint_definition)
            if len(coef) > 0:
                constraint_definition.set_linear_coefficients(coef)
                
            self.model.solver.update()
        return constraint_definition
    
    
    def all_variables(self):
        '''Quantify the variables in the class
        '''
        vars = {}
        for child in self.childpkgs:
            for type in child.variables:
                vars[type] = child.variables[type]

        for type in self.variables:
            vars[type] = self.variables[type]
        return vars
    
    
    def all_constraints(self):
        '''Quantify the constraints in the class
        '''
        consts = {}
        for child in self.childpkgs:
            for type in child.constraints:
                consts[type] = child.constraints[type]

        for type in self.constraints:
            consts[type] = self.constraints[type]
        return consts
    
    
    
# ---------------------------------------------- Revbin -------------------------------------------------

# The base class for FBA packages is inherited
class RevBinPkg(BaseFBAPkg):
    '''Note -> revbin = 0 describes reversibility, while revbin = 1 describes forward progression
    '''
    def __init__(self, model):
        '''Redefining the inherited __init__ function
        'model' (COBRA object): The COBRApy FBA model
        '''
        BaseFBAPkg.__init__(self, model = model, name = "reversible binary", variable_types = {"revbin":"reaction"}, constraint_types = {"revbinF":"reaction", "revbinR":"reaction"})
    

    def build_constraint(self, object):
        '''Build constraints through the inherited function and the calculated coefficient fluxes
        'object' (Python object, string): The variable name when the name is defined
        '''
        #-1000 * revbin(i) + forv(i) <= 0
        BaseFBAPkg.build_constraint(self, type = "revbinF", lower_bound = None, upper_bound = 1000, coef = {self.variables["revbin"][object.id]:-1000, object.forward_variable:1}, object = object)
        
        #1000 * revbin(i) + revv(i) <= 1000
        return BaseFBAPkg.build_constraint(self, type = "revbinR", lower_bound = None, upper_bound = 1000, coef = {self.variables["revbin"][object.id]:1000, object.reverse_variable:0}, object = object)
    
    
    def build_package(self, filter = None):
        '''Build variables and constraints through the inherited function
        'filter' (Python object, list): The accepted list of reactions that will be built into the model
        '''
        for reaction in self.model.reactions:
            # Unfiltered reactions are constructed through the aforementioned functions
            if filter == None or reaction.id in filter:
                BaseFBAPkg.build_variable(self, type = "revbin", lower_bound = 0, upper_bound = 1, vartype = "binary", object = reaction)
                self.build_constraint(reaction)
                
                
# ------------------------------------------ Simple Thermo package ------------------------------------------                

# The base class for FBA packages is inherited
class SimpleThermoPkg(BaseFBAPkg):
    def __init__(self, model):
        '''Redefining the inherited __init__ function
        'model' (COBRA object): The COBRApy FBA model
        '''
        BaseFBAPkg.__init__(self, model = model, name = "simple thermo", variable_types = {"potential":"metabolite"}, constraint_types = {"thermo":"reaction"})
        self.childpkgs["reversible binary"] = RevBinPkg(model)
    

    def build_constraint(self, object):# Gibbs: dg = Sum(st(i,j)*p(j))
        '''Build constraints through the inherited function and the calculated variable coeffiients 
        'object' (Python object, string): The variable name when the name is defined
        '''
        # 0 <= 1000*revbin(i) + Sum(st(i,j)*p(j)) <= 1000
        coef = {self.childpkgs["reversible binary"].variables["revbin"][object.id] : 1000}
        for metabolite in object.metabolites:
            coef[self.variables["potential"][metabolite.id]] = object.metabolites[metabolite]
            
        return BaseFBAPkg.build_constraint(self,type = "thermo", lower_bound = 0, upper_bound = 1000, coef = coef, object = object)

        
    def build_package(self, filter = None):
        '''Build variables and constraints through the inherited function
        'filter' (Python object, list): The accepted list of reactions that will be built into the model
        '''
        self.childpkgs["reversible binary"].build_package(filter)
        for metabolite in self.model.metabolites:
            BaseFBAPkg.build_variable(self, type = "potential", lower_bound = 0, upper_bound = 1000, vartype = "continuous", object = metabolite)
            
        for reaction in self.model.reactions:
            # Unfiltered reactions are constructed through the aforementioned functions
            if filter == None or reaction.id in filter:
                self.build_constraint(reaction)
                
                
# ------------------------------------------ Full Thermo package ------------------------------------------

# The base class for FBA packages is inherited
class FullThermoPkg(BaseFBAPkg):
    def __init__(self, model):
        '''Redefining the inherited __init__ function
        'model' (COBRA object): The COBRApy FBA model
        '''
        BaseFBAPkg.__init__(self, model = model, name = "full thermo", variable_types = {"llnconc":"metabolite"}, constraint_types = {"potentialc":"metabolite"})
        self.childpkgs["simple thermo"] = SimpleThermoPkg(model)
    
        self.validate_parameters(params = parameters, required = [], defaults = {
            "default_max_conc": 20,
            "default_min_conc": 0.001,
            "custom_concentration_constraints": {"cpd000027_e0": ["10","10"]},
            "compartment_potential": {"c0": ,}
        })
    
    
    def build_variable(self, object):
        '''Build variables through the inherited function
        'object' (Python object, string): The variable name when the name is defined
        '''
        if object.id in self.parameters["custom_concentration_constraints"].keys():
            lb = ln(self.parameters["custom_concentration_constraints"][object.id][0])
            ub = ln(self.parameters["custom_concentration_constraints"][object.id][1])
        else:
            lb = ln(self.parameters["default_min_conc"])
            ub = ln(self.parameters["default_max_conc"])
            
        return BaseFBAPkg.build_variable(self, type = "lnconc", lower_bound = lb, upper_bound = ub, vartype = "continuous", object = object)
    
    
    def build_constraint(self, object):
        '''Build constraints through the inherited function and the calculated variable coeffiients 
        'object' (Python object, string): The variable name when the name is defined
        '''
        # Equation 14 in the TMFA paper, with the addition of the (charge * compartment_potential) term?
        # potential(i) (KJ/mol) = deltaG^naut(i) (KJ/mol) + R * T(K) * Sum(x_i * lnconc(i * gamma)) + charge(i) * compartment_potential
        constant = ?
        coef = {self.childpkgs["potential"].variables["revbin"][object.id] : 1000}
        return BaseFBAPkg.build_constraint(self, type = "thermo", lower_bound = constant, upper_bound = constant, coef = coef, object = object)

        
    def build_package(self, parameters):
        '''Create the final model package
        'parameters' (Python object, dictionary): A dictionary of parameters and values
        Notes - The concentrations are expressed in millimolar
        '''        
        # Extend the custom concentration for H+, CO2, O2
        self.childpkgs["simple thermo"].build_package(filter)
        
        # The concentration variable and potential constraint are built
        for metabolite in self.model.metabolites:
            self.build_variable(metabolite)
            self.build_constraint(reaction)

# Brainstorming

In [2]:
dictionary = {'a':1, 'b':2}
require = {'a':2, 'c':2}
set_1 = set(require) - set(dictionary)
print(set_1)

{'c'}


In [5]:
defaults = {"custom_concentration_constraints": {"cpd000027_e0": ["10","10"],
                                                'cpd000033_c0': ['3', '4']}}
print(defaults["custom_concentration_constraints"]['cpd000033_c0'][0])

3
